# 1. LGDE.com 일별 지표생성 실습 2일차 (정답)

#### 주피터 노트북 단축키 (Windows 환경)
| 단축키 | 설명 | 기타 |
| --- | --- | --- |
| Alt+Enter | 현재 셀 실행 + 다음 셀 추가 | 초기 개발 시에 주로 사용 |
| Shift+Enter | 현재 셀 실행 + 다음 셀 이동 | 전체 테스트 시에 주로 사용 |
| Ctrl+Enter | 현재 셀 실행 + 이동 안함 | 하나씩 점검 혹은 디버깅 시에 사용 |
| Ctrl+/ | 주석 적용 및 해제 | Shift 키로 여러 줄을 선택하고 주석 및 해제 사용 |
| Ctrl+s | 전체 저장 | - |

### 주피터 노트북 유의사항
* 모든 셀은 Code, Markdown, Raw 3가지 유형이 존재하며, **파이썬 코드 실행**은 반드시 ***Code*** 모드에서 수행되어야 합니다
* 현재 셀의 실행이 무한 루프에 빠지거나 너무 오래 걸리는 경우 상단 Menu 에서 ***Kernel - Interrupt Kernel*** 메뉴를 통해 현재 셀의 작업만 중지할 수 있습니다
* 메모리 혹은 다양한 이슈로 인해 제대로 동작하지 않는 경우에는 상단 Menu 에서 ***Kernel - Restart Kernel..*** 메뉴를 통해 재시작할 수 있습니다


## 5. 수집된 데이터 탐색
### 5-1. 스파크 세션 생성

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

21/09/05 12:44:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/09/05 12:44:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
user26 = spark.read.parquet("user/20201026")
user26.printSchema()
user26.show(truncate=False)
display(user26)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)



+----+----------+--------+--------+
|u_id|u_name    |u_gender|u_signup|
+----+----------+--------+--------+
|1   |정휘센    |남      |20201025|
|2   |김싸이언  |남      |20201025|
|3   |박트롬    |여      |20201025|
|4   |청소기    |남      |20201025|
|5   |유코드제로|여      |20201025|
|6   |윤디오스  |남      |20201026|
|7   |임모바일  |남      |20201026|
|8   |조노트북  |여      |20201026|
|9   |최컴퓨터  |남      |20201026|
+----+----------+--------+--------+



u_id,u_name,u_gender,u_signup
1,정휘센,남,20201025
2,김싸이언,남,20201025
3,박트롬,여,20201025
4,청소기,남,20201025
5,유코드제로,여,20201025
6,윤디오스,남,20201026
7,임모바일,남,20201026
8,조노트북,여,20201026
9,최컴퓨터,남,20201026


In [3]:
purchase26 = spark.read.parquet("purchase/20201026")
purchase26.printSchema()
purchase26.show(truncate=False)
display(purchase26)

root
 |-- p_time: string (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)

+----------+-----+----+-----------+--------+
|p_time    |p_uid|p_id|p_name     |p_amount|
+----------+-----+----+-----------+--------+
|1603571550|1    |2000|LG DIOS    |2000000 |
|1603614755|1    |2000|LG Gram    |1800000 |
|1603593500|2    |2001|LG Cyon    |1400000 |
|1603572155|3    |2002|LG TV      |1000000 |
|1603585955|5    |2004|LG Gram    |3500000 |
|1603586155|5    |2004|LG TV      |2500000 |
|1603651550|1    |2001|LG Cyon    |1400000 |
|1603652155|5    |2002|LG TV      |1000000 |
|1603674500|6    |2003|LG Computer|4500000 |
|1603665955|7    |2004|LG Gram    |3500000 |
|1603666155|9    |2004|LG TV      |2500000 |
+----------+-----+----+-----------+--------+



p_time,p_uid,p_id,p_name,p_amount
1603571550,1,2000,LG DIOS,2000000
1603614755,1,2000,LG Gram,1800000
1603593500,2,2001,LG Cyon,1400000
1603572155,3,2002,LG TV,1000000
1603585955,5,2004,LG Gram,3500000
1603586155,5,2004,LG TV,2500000
1603651550,1,2001,LG Cyon,1400000
1603652155,5,2002,LG TV,1000000
1603674500,6,2003,LG Computer,4500000
1603665955,7,2004,LG Gram,3500000


In [4]:
access26 = spark.read.option("inferSchema", "true").json("access/20201026")
access26.printSchema()
access26.show(truncate=False)
display(access26)

root
 |-- a_id: string (nullable = true)
 |-- a_tag: string (nullable = true)
 |-- a_time: long (nullable = true)
 |-- a_timestamp: string (nullable = true)
 |-- a_uid: long (nullable = true)

+------+------+----------+-----------------------+-----+
|a_id  |a_tag |a_time    |a_timestamp            |a_uid|
+------+------+----------+-----------------------+-----+
|logout|access|1603647200|2020-10-26 02:33:20.000|1    |
|logout|access|1603650200|2020-10-26 03:23:20.000|2    |
|logout|access|1603659200|2020-10-26 05:53:20.000|3    |
|logout|access|1603664200|2020-10-26 07:16:40.000|4    |
|logout|access|1603669500|2020-10-26 08:45:00.000|5    |
|login |access|1603645200|2020-10-26 02:00:00.000|1    |
|login |access|1603649200|2020-10-26 03:06:40.000|2    |
|login |access|1603653200|2020-10-26 04:13:20.000|2    |
|login |access|1603657200|2020-10-26 05:20:00.000|3    |
|login |access|1603660200|2020-10-26 06:10:00.000|4    |
|login |access|1603664500|2020-10-26 07:21:40.000|4    |
|login |a

a_id,a_tag,a_time,a_timestamp,a_uid
logout,access,1603647200,2020-10-26 02:33:20.000,1
logout,access,1603650200,2020-10-26 03:23:20.000,2
logout,access,1603659200,2020-10-26 05:53:20.000,3
logout,access,1603664200,2020-10-26 07:16:40.000,4
logout,access,1603669500,2020-10-26 08:45:00.000,5
login,access,1603645200,2020-10-26 02:00:00.000,1
login,access,1603649200,2020-10-26 03:06:40.000,2
login,access,1603653200,2020-10-26 04:13:20.000,2
login,access,1603657200,2020-10-26 05:20:00.000,3
login,access,1603660200,2020-10-26 06:10:00.000,4


### 5-2. 수집된 고객, 매출 및 접속 임시 테이블 생성

In [5]:
user26.createOrReplaceTempView("user26")
purchase26.createOrReplaceTempView("purchase26")
access26.createOrReplaceTempView("access26")
spark.sql("show tables '*26'")

database,tableName,isTemporary
,access26,true
,purchase26,true
,user26,true


### 5-3. SparkSQL을 이용하여 테이블 별 데이터프레임 생성하기

In [6]:
u_signup_condition = "u_signup >= '20201026' and u_signup < '20201027'"
user = spark.sql("select u_id, u_name, u_gender from user26").where(u_signup_condition)
user.createOrReplaceTempView("user")

p_time_condition = "p_time >= '2020-10-26 00:00:00' and p_time < '2020-10-27 00:00:00'"
purchase = spark.sql("select from_unixtime(p_time) as p_time, p_uid, p_id, p_name, p_amount from purchase26").where(p_time_condition)
purchase.createOrReplaceTempView("purchase")

access = spark.sql("select a_id, a_tag, a_timestamp, a_uid from access26")
access.createOrReplaceTempView("access")

spark.sql("show tables")

database,tableName,isTemporary
,access,true
,access26,true
,purchase,true
,purchase26,true
,user,true
,user26,true


### 5-4. 생성된 테이블을 SQL 문을 이용하여 탐색하기

In [7]:
spark.sql("describe user")
groupByCount = "select u_gender, count(1) from user group by u_gender"
spark.sql(groupByCount)

u_gender,count(1)
여,1
남,3


In [8]:
spark.sql("describe purchase")
selectClause = "select min(p_amount), max(p_amount) from purchase"
spark.sql(selectClause)

min(p_amount),max(p_amount)
1000000,4500000


In [9]:
spark.sql("describe access")
countTop="select a_uid, count(1) as a_count from access where a_id = 'login' group by a_uid order by a_count desc"
spark.sql(countTop)

a_uid,a_count
2,2
4,2
6,1
7,1
3,1
1,1
8,1
5,1
9,1


## 6. 기본 지표 생성
### 6-1. DAU (Daily Activer User) 지표를 생성하세요

In [10]:
display(access)
distinctAccessUser = "select count(distinct a_uid) as DAU from access"
dau = spark.sql(distinctAccessUser)
display(dau)

a_id,a_tag,a_timestamp,a_uid
logout,access,2020-10-26 02:33:20.000,1
logout,access,2020-10-26 03:23:20.000,2
logout,access,2020-10-26 05:53:20.000,3
logout,access,2020-10-26 07:16:40.000,4
logout,access,2020-10-26 08:45:00.000,5
login,access,2020-10-26 02:00:00.000,1
login,access,2020-10-26 03:06:40.000,2
login,access,2020-10-26 04:13:20.000,2
login,access,2020-10-26 05:20:00.000,3
login,access,2020-10-26 06:10:00.000,4


DAU
9


### 6-2. DPU (Daily Paying User) 지표를 생성하세요

In [11]:
display(purchase)
distinctPayingUser = "select count(distinct p_uid) as PU from purchase"
pu = spark.sql(distinctPayingUser)
display(pu)

p_time,p_uid,p_id,p_name,p_amount
2020-10-26 03:45:50,1,2001,LG Cyon,1400000
2020-10-26 03:55:55,5,2002,LG TV,1000000
2020-10-26 10:08:20,6,2003,LG Computer,4500000
2020-10-26 07:45:55,7,2004,LG Gram,3500000
2020-10-26 07:49:15,9,2004,LG TV,2500000


PU
5


### 6-3. DR (Daily Revenue) 지표를 생성하세요

In [12]:
display(purchase)
sumOfDailyRevenue = "select sum(p_amount) as DR from purchase"
dr = spark.sql(sumOfDailyRevenue)
display(dr)

p_time,p_uid,p_id,p_name,p_amount
2020-10-26 03:45:50,1,2001,LG Cyon,1400000
2020-10-26 03:55:55,5,2002,LG TV,1000000
2020-10-26 10:08:20,6,2003,LG Computer,4500000
2020-10-26 07:45:55,7,2004,LG Gram,3500000
2020-10-26 07:49:15,9,2004,LG TV,2500000


DR
12900000


### 6-4. ARPU (Average Revenue Per User) 지표를 생성하세요

In [13]:
v_dau = dau.collect()[0]["DAU"]
v_pu = pu.collect()[0]["PU"]
v_dr = dr.collect()[0]["DR"]

print("ARPU : {}".format(v_dr / v_dau))

ARPU : 1433333.3333333333


### 6-5. ARPPU (Average Revenue Per Paying User) 지표를 생성하세요

In [14]:
print("ARPPU : {}".format(v_dr / v_pu))

ARPPU : 2580000.0


## 7. 고급 지표 생성

> 어제 디멘젼을 가져와서 정상 조회 되는 지 확인합니다

In [15]:
yesterday_dimension="dimension/dt=20201025"
yesterday = spark.read.parquet(yesterday_dimension)
yesterday.printSchema()
display(yesterday)

root
 |-- d_uid: long (nullable = true)
 |-- d_name: string (nullable = true)
 |-- d_gender: string (nullable = true)
 |-- d_acount: long (nullable = true)
 |-- d_pamount: long (nullable = true)
 |-- d_pcount: long (nullable = true)
 |-- d_first_purchase: string (nullable = true)



d_uid,d_name,d_gender,d_acount,d_pamount,d_pcount,d_first_purchase
5,유코드제로,여,2,6000000,2,2020-10-25 09:32:35
2,김싸이언,남,3,1400000,1,2020-10-25 11:38:20
1,정휘센,남,2,3800000,2,2020-10-25 05:32:30
3,박트롬,여,2,1000000,1,2020-10-25 05:42:35
4,청소기,남,3,0,0,null


### 7-1. 전체 고객의 ID 추출

In [16]:
yesterday_uids = yesterday.select("d_uid")
today_uids = access.select("a_uid")

joinCondition = "<어제 uid 와 오늘 uid 값이 일치하는 조건>"
joinCondition = yesterday_uids.d_uid == today_uids.a_uid

joinHow = "<어제와 오늘 고객정보를 추가하는 방식>"
joinHow = "full_outer"

all_uids = (
    yesterday_uids.join(today_uids, joinCondition, joinHow)
    .withColumn("uid", when(yesterday.d_uid.isNull(), access.a_uid).otherwise(yesterday.d_uid))
    .select("uid").distinct()
)

all_uids.printSchema()
display(all_uids.orderBy("uid"))

root
 |-- uid: long (nullable = true)



uid
1
2
3
4
5
6
7
8
9


In [17]:
uid1 = yesterday.select("d_uid").withColumnRenamed("d_uid", "a_uid")
uid2 = access.select("a_uid")
all_uids = uid1.union(uid2).distinct()
all_uids.printSchema()
display(all_uids)

root
 |-- a_uid: long (nullable = true)



a_uid
4
6
7
8
3
5
2
1
9


### 7-2. 어제 디멘젼 정보와 결합

In [18]:
joinCondition = "uid 와 yesterday 의 uid 가 일치하는 조건"
joinCondition = all_uids.a_uid == yesterday.d_uid
joinHow = "left_outer"

# uid 에서 생성된 id 값이 모든 user_id 이므로, drop 함수를 통해 기존의 d_uid 는 제거하고, withColumnRenamed 함수를 통해, uid 를 d_uid 로 변경 합니다
uids = (
    all_uids.join(yesterday, joinCondition, joinHow)
    .drop("d_uid")
    .withColumnRenamed("a_uid", "d_uid")
    .sort(asc("d_uid"))
)

uids.printSchema()
display(uids)

root
 |-- d_uid: long (nullable = true)
 |-- d_name: string (nullable = true)
 |-- d_gender: string (nullable = true)
 |-- d_acount: long (nullable = true)
 |-- d_pamount: long (nullable = true)
 |-- d_pcount: long (nullable = true)
 |-- d_first_purchase: string (nullable = true)



d_uid,d_name,d_gender,d_acount,d_pamount,d_pcount,d_first_purchase
1,정휘센,남,2,3800000,2,2020-10-25 05:32:30
2,김싸이언,남,3,1400000,1,2020-10-25 11:38:20
3,박트롬,여,2,1000000,1,2020-10-25 05:42:35
4,청소기,남,3,0,0,null
5,유코드제로,여,2,6000000,2,2020-10-25 09:32:35
6,null,null,null,null,null,null
7,null,null,null,null,null,null
8,null,null,null,null,null,null
9,null,null,null,null,null,null


### 7-3. 이름과 성별을 결합

In [19]:
user.show()
uids.show()

+----+--------+--------+
|u_id|  u_name|u_gender|
+----+--------+--------+
|   6|윤디오스|      남|
|   7|임모바일|      남|
|   8|조노트북|      여|
|   9|최컴퓨터|      남|
+----+--------+--------+

+-----+----------+--------+--------+---------+--------+-------------------+
|d_uid|    d_name|d_gender|d_acount|d_pamount|d_pcount|   d_first_purchase|
+-----+----------+--------+--------+---------+--------+-------------------+
|    1|    정휘센|      남|       2|  3800000|       2|2020-10-25 05:32:30|
|    2|  김싸이언|      남|       3|  1400000|       1|2020-10-25 11:38:20|
|    3|    박트롬|      여|       2|  1000000|       1|2020-10-25 05:42:35|
|    4|    청소기|      남|       3|        0|       0|               null|
|    5|유코드제로|      여|       2|  6000000|       2|2020-10-25 09:32:35|
|    6|      null|    null|    null|     null|    null|               null|
|    7|      null|    null|    null|     null|    null|               null|
|    8|      null|    null|    null|     null|    null|               null|
|    9

In [20]:
# 실습25) d_name 이 null 이면 u_name 을 사용하고 그렇지 않으면 d_name 을 사용합니다
exprName = expr("case when d_name is null then u_name else d_name end")

# 실습26) d_gender 이 null 이면 u_gender 을 사용하고 그렇지 않으면 d_gender 을 사용합니다
exprGender = expr("case when d_gender is null then u_gender else d_gender end")

dim1 = (
    uids.join(user, uids.d_uid == user.u_id, "left_outer")
    .withColumn("name", exprName)
    .withColumn("gender", exprGender)
    .drop("d_name", "d_gender", "u_id", "u_name", "u_gender")
    .withColumnRenamed("name", "d_name")
    .withColumnRenamed("gender", "d_gender")
).orderBy(asc("d_uid"))

display(dim1)

d_uid,d_acount,d_pamount,d_pcount,d_first_purchase,d_name,d_gender
1,2,3800000,2,2020-10-25 05:32:30,정휘센,남
2,3,1400000,1,2020-10-25 11:38:20,김싸이언,남
3,2,1000000,1,2020-10-25 05:42:35,박트롬,여
4,3,0,0,null,청소기,남
5,2,6000000,2,2020-10-25 09:32:35,유코드제로,여
6,null,null,null,null,윤디오스,남
7,null,null,null,null,임모바일,남
8,null,null,null,null,조노트북,여
9,null,null,null,null,최컴퓨터,남


### 7-4. 숫자 필드에 널값은 0으로 기본값을 넣어줍니다

In [21]:
# 실습27) d_acount, d_pamount, d_pcount 필드의 기본값을 0으로 넣도록 작성합니다
fillDefaultValue = { "d_acount":0, "d_pamount":0, "d_pcount":0 }

dim2 = dim1.na.fill(fillDefaultValue)
display(dim2)

d_uid,d_acount,d_pamount,d_pcount,d_first_purchase,d_name,d_gender
1,2,3800000,2,2020-10-25 05:32:30,정휘센,남
2,3,1400000,1,2020-10-25 11:38:20,김싸이언,남
3,2,1000000,1,2020-10-25 05:42:35,박트롬,여
4,3,0,0,null,청소기,남
5,2,6000000,2,2020-10-25 09:32:35,유코드제로,여
6,0,0,0,null,윤디오스,남
7,0,0,0,null,임모바일,남
8,0,0,0,null,조노트북,여
9,0,0,0,null,최컴퓨터,남


### 7-5. 접속횟수를 결합

In [22]:
access_sum = spark.sql("select a_uid, count(a_uid) as a_count from access where a_id = 'login' group by a_uid")
access_sum.printSchema()
access_sum.show()
dim2.printSchema()

# 실습28) 오늘 접속 수치(a_count)가 null 이면 디멘젼의 d_acount 값을 사용하고, 그렇지 않으면 d_acount + a_count 를 사용합니다
sumOfAccess = expr("a_count")
sumOfAccess = expr("case when a_count is null then d_acount else d_acount + a_count end")

dim3 = (
    dim2.join(access_sum, dim2.d_uid == access_sum.a_uid, "left_outer")
    .withColumn("sum_of_access", sumOfAccess)
    .drop("a_uid", "a_count", "d_acount")
    .withColumnRenamed("sum_of_access", "d_acount")
).orderBy(asc("d_uid"))
dim3.printSchema()
display(dim3)

root
 |-- a_uid: long (nullable = true)
 |-- a_count: long (nullable = false)

+-----+-------+
|a_uid|a_count|
+-----+-------+
|    4|      2|
|    6|      1|
|    7|      1|
|    8|      1|
|    3|      1|
|    2|      2|
|    5|      1|
|    1|      1|
|    9|      1|
+-----+-------+

root
 |-- d_uid: long (nullable = true)
 |-- d_acount: long (nullable = false)
 |-- d_pamount: long (nullable = false)
 |-- d_pcount: long (nullable = false)
 |-- d_first_purchase: string (nullable = true)
 |-- d_name: string (nullable = true)
 |-- d_gender: string (nullable = true)

root
 |-- d_uid: long (nullable = true)
 |-- d_pamount: long (nullable = false)
 |-- d_pcount: long (nullable = false)
 |-- d_first_purchase: string (nullable = true)
 |-- d_name: string (nullable = true)
 |-- d_gender: string (nullable = true)
 |-- d_acount: long (nullable = true)



d_uid,d_pamount,d_pcount,d_first_purchase,d_name,d_gender,d_acount
1,3800000,2,2020-10-25 05:32:30,정휘센,남,3
2,1400000,1,2020-10-25 11:38:20,김싸이언,남,5
3,1000000,1,2020-10-25 05:42:35,박트롬,여,3
4,0,0,null,청소기,남,5
5,6000000,2,2020-10-25 09:32:35,유코드제로,여,3
6,0,0,null,윤디오스,남,1
7,0,0,null,임모바일,남,1
8,0,0,null,조노트북,여,1
9,0,0,null,최컴퓨터,남,1


### 7-6. 매출횟수 및 매출을 결합

In [24]:
display(dim3)
dim3.printSchema()

purchase_sum = spark.sql("select p_uid, sum(p_amount) as pamount, count(p_uid) as pcount from purchase group by p_uid")

display(purchase_sum)
purchase_sum.printSchema()

d_uid,d_pamount,d_pcount,d_first_purchase,d_name,d_gender,d_acount
1,3800000,2,2020-10-25 05:32:30,정휘센,남,3
2,1400000,1,2020-10-25 11:38:20,김싸이언,남,5
3,1000000,1,2020-10-25 05:42:35,박트롬,여,3
4,0,0,null,청소기,남,5
5,6000000,2,2020-10-25 09:32:35,유코드제로,여,3
6,0,0,null,윤디오스,남,1
7,0,0,null,임모바일,남,1
8,0,0,null,조노트북,여,1
9,0,0,null,최컴퓨터,남,1


root
 |-- d_uid: long (nullable = true)
 |-- d_pamount: long (nullable = false)
 |-- d_pcount: long (nullable = false)
 |-- d_first_purchase: string (nullable = true)
 |-- d_name: string (nullable = true)
 |-- d_gender: string (nullable = true)
 |-- d_acount: long (nullable = true)



p_uid,pamount,pcount
5,1000000,1
7,3500000,1
1,1400000,1
6,4500000,1
9,2500000,1


root
 |-- p_uid: integer (nullable = true)
 |-- pamount: long (nullable = true)
 |-- pcount: long (nullable = false)



In [25]:
# 실습29) 매출 금액의 합(d_pamount + pamount)을 구합니다
sumOfAmount = expr("case when pamount is null then d_pamount else d_pamount + pamount end")

# 실습30) 매출 빈도의 합(d_pcount + pcount)을 구합니다
sumOfCount = expr("case when pcount is null then d_pcount else d_pcount + pcount end")

dim4 = (
    dim3.join(purchase_sum, dim3.d_uid == purchase_sum.p_uid, "left")
    .withColumn("sum_of_amount", sumOfAmount)
    .withColumn("sum_of_count", sumOfCount)
    .drop("p_uid", "d_pamount", "d_pcount", "pamount", "pcount")
    .withColumnRenamed("sum_of_amount", "d_pamount")
    .withColumnRenamed("sum_of_count", "d_pcount")
).orderBy(asc("d_uid"))

dim4.printSchema()
display(dim4)

root
 |-- d_uid: long (nullable = true)
 |-- d_first_purchase: string (nullable = true)
 |-- d_name: string (nullable = true)
 |-- d_gender: string (nullable = true)
 |-- d_acount: long (nullable = true)
 |-- d_pamount: long (nullable = true)
 |-- d_pcount: long (nullable = true)



d_uid,d_first_purchase,d_name,d_gender,d_acount,d_pamount,d_pcount
1,2020-10-25 05:32:30,정휘센,남,3,5200000,3
2,2020-10-25 11:38:20,김싸이언,남,5,1400000,1
3,2020-10-25 05:42:35,박트롬,여,3,1000000,1
4,null,청소기,남,5,0,0
5,2020-10-25 09:32:35,유코드제로,여,3,7000000,3
6,null,윤디오스,남,1,4500000,1
7,null,임모바일,남,1,3500000,1
8,null,조노트북,여,1,0,0
9,null,최컴퓨터,남,1,2500000,1


### 7-7. 최초 구매 일자를 결합

In [26]:
# 실습31) 하루에 여러번 구매가 있을 수 있으므로 group by p_uid 집계를 통해 가장 먼저 구매한 정보 즉, min(p_time)함수를 통해 일시를 선택합니다
selectFirstPurchaseTime = "select p_uid, min(p_time) as p_time from purchase group by p_uid"

first_purchase = spark.sql(selectFirstPurchaseTime)
first_purchase.printSchema()
first_purchase.show()

# 실습32) 디멘젼의 최초구매일(d_first_purchase)이 null 이라면 p_time 을 사용하고 그렇지 않으면 d_first_purchase 를 사용합니다
exprFirstPurchase = expr("case when d_first_purchase is null then p_time else d_first_purchase end")

dimension = (
    dim4.join(first_purchase, dim4.d_uid == first_purchase.p_uid, "left")
    .withColumn("first_purchase", exprFirstPurchase)
    .drop("p_uid", "p_time", "d_first_purchase")
    .withColumnRenamed("first_purchase", "d_first_purchase")
).orderBy("d_uid")

dimension.printSchema()
display(dimension)

root
 |-- p_uid: integer (nullable = true)
 |-- p_time: string (nullable = true)

+-----+-------------------+
|p_uid|             p_time|
+-----+-------------------+
|    5|2020-10-26 03:55:55|
|    7|2020-10-26 07:45:55|
|    1|2020-10-26 03:45:50|
|    6|2020-10-26 10:08:20|
|    9|2020-10-26 07:49:15|
+-----+-------------------+

root
 |-- d_uid: long (nullable = true)
 |-- d_name: string (nullable = true)
 |-- d_gender: string (nullable = true)
 |-- d_acount: long (nullable = true)
 |-- d_pamount: long (nullable = true)
 |-- d_pcount: long (nullable = true)
 |-- d_first_purchase: string (nullable = true)



d_uid,d_name,d_gender,d_acount,d_pamount,d_pcount,d_first_purchase
1,정휘센,남,3,5200000,3,2020-10-25 05:32:30
2,김싸이언,남,5,1400000,1,2020-10-25 11:38:20
3,박트롬,여,3,1000000,1,2020-10-25 05:42:35
4,청소기,남,5,0,0,null
5,유코드제로,여,3,7000000,3,2020-10-25 09:32:35
6,윤디오스,남,1,4500000,1,2020-10-26 10:08:20
7,임모바일,남,1,3500000,1,2020-10-26 07:45:55
8,조노트북,여,1,0,0,null
9,최컴퓨터,남,1,2500000,1,2020-10-26 07:49:15


### 7-8. 신규유저를 계산

In [27]:
# 어제와 오늘의 디멘젼 테이블을 이용해 오늘 처음 접속한 신규 유저를 구합니다
today_uids = dimension.select("d_uid")
yesterday_uids = yesterday.select("d_uid")

nu = today_uids.subtract(yesterday_uids)
nu.printSchema()
nu.show()
v_nu = nu.count()
print("NU: {}".format(v_nu))

root
 |-- d_uid: long (nullable = true)

+-----+
|d_uid|
+-----+
|    6|
|    7|
|    8|
|    9|
+-----+

NU: 4


### 7-9. 생성된 디멘젼을 저장소에 저장합니다

In [28]:
dimension.printSchema()
target_dir="dimension/dt=20201026"
dimension.write.mode("overwrite").parquet(target_dir)

root
 |-- d_uid: long (nullable = true)
 |-- d_name: string (nullable = true)
 |-- d_gender: string (nullable = true)
 |-- d_acount: long (nullable = true)
 |-- d_pamount: long (nullable = true)
 |-- d_pcount: long (nullable = true)
 |-- d_first_purchase: string (nullable = true)



### 7-10. 생성된 디멘젼을 다시 읽어서 출력합니다

In [29]:
newDimension = spark.read.parquet(target_dir)
newDimension.printSchema()
display(newDimension)

root
 |-- d_uid: long (nullable = true)
 |-- d_name: string (nullable = true)
 |-- d_gender: string (nullable = true)
 |-- d_acount: long (nullable = true)
 |-- d_pamount: long (nullable = true)
 |-- d_pcount: long (nullable = true)
 |-- d_first_purchase: string (nullable = true)



d_uid,d_name,d_gender,d_acount,d_pamount,d_pcount,d_first_purchase
9,최컴퓨터,남,1,2500000,1,2020-10-26 07:49:15
7,임모바일,남,1,3500000,1,2020-10-26 07:45:55
8,조노트북,여,1,0,0,null
3,박트롬,여,3,1000000,1,2020-10-25 05:42:35
4,청소기,남,5,0,0,null
1,정휘센,남,3,5200000,3,2020-10-25 05:32:30
2,김싸이언,남,5,1400000,1,2020-10-25 11:38:20
5,유코드제로,여,3,7000000,3,2020-10-25 09:32:35
6,윤디오스,남,1,4500000,1,2020-10-26 10:08:20
